In [10]:
import joblib
import pandas as pd
import numpy as np

# Load the models
xgboost_model = joblib.load('model_xgboost.pkl')
linear_model = joblib.load('model_linear_ffs.pkl')

# Prepare input data
# Note: The models expect log-transformed ROS as output

def safe_log(x):
    """Safe logarithm transformation: sign(x) * ln(|x| + 1)"""
    return np.sign(x) * np.log(np.abs(x) + 1)

# Example input data for XGBoost model
xgboost_features = ['duration_p', '8_ny_fir_p', 'sW_100_av', '3_8y_fir_p', 'f_start', 'HDW_av']
                    
input_data_xgb = pd.DataFrame({
    'duration_p': [1.5],
    '3_8y_fir_p': [10.1],
    '8_ny_fir_p': [59.1],
    'sW_100_av': [0.21],
    'f_start': [1593],
    'HDW_av': [27268]
})

# Example input data for Linear model
linear_features = ['HDW_av', 'wv_850_av', 'duration_p', 'Cape_av_log', 'gT_8_7_av',
        '3_8y_fir_p_log', 'DC_12h_av_log']

input_data_linear = pd.DataFrame({
    'duration_p': [1.5],                    
    '3_8y_fir_p_log': [safe_log(10.1)],     
    'HDW_av': [27268],                      
    'wv_850_av': [23.5],                    
    'Cape_av_log': [safe_log(92.3)],        
    'gT_8_7_av': [-7.46],                   
    'DC_12h_av_log': [safe_log(700.2)],    
})

# Make predictions (these will be in log space)
ros_log_xgb = xgboost_model.predict(input_data_xgb[xgboost_features])
ros_log_linear = linear_model.predict(input_data_linear[linear_features])

# Convert to linear scale
ros_linear_xgb = np.exp(ros_log_xgb)
ros_linear_linear = np.exp(ros_log_linear)

print(f"XGBoost prediction: {ros_linear_xgb[0]:.1f} m/h")
print(f"Linear model prediction: {ros_linear_linear[0]:.1f} m/h")

XGBoost prediction: 1211.0 m/h
Linear model prediction: 641.2 m/h
